In [27]:
import open3d as o3d
import numpy as np
from open3d import JVisualizer
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import copy 
import os

In [28]:
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)
    pcd_down, ind = pcd_down.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    # orient normals so the ground is facing the same way
    pcd_down.orient_normals_to_align_with_direction()
    pcd_down = remove_ground_points(pcd_down, 0.4)

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

In [29]:
def prepare_dataset(voxel_size):
    print(":: Load two point clouds")
    source = o3d.io.read_point_cloud("/home/zak/senior_design/ros_livox/maps/fri_mar_12/corner.pcd")
    target = o3d.io.read_point_cloud("/home/zak/senior_design/ros_livox/maps/mon_mar_15_after_snow_2/corner.pcd")
    trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    draw_registration_result(source, target, np.identity(4))

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

In [30]:
def remove_ground_points(pcd, z_threshold):
    
    pop_list = []
    for i, normal in enumerate(pcd.normals):
        if normal[2] > z_threshold:
            pop_list.append(i)
    
    print("Number of points to be removed: ", len(pop_list))
    pop_list.sort(reverse=True)
    for item in pop_list:
        pcd.normals.pop(item)
        pcd.points.pop(item)
    
    return pcd

In [31]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4559,
                                      front=[0.6452, -0.3036, -0.7011],
                                      lookat=[1.9892, 2.0208, 1.8945],
                                      up=[-0.2779, -0.9482, 0.1556])

In [32]:
voxel_size = 0.4
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(
    voxel_size)

:: Load two point clouds
:: Downsample with a voxel size 0.400.
:: Estimate normal with search radius 0.800.
Number of points to be removed:  14050
:: Compute FPFH feature with search radius 2.000.
:: Downsample with a voxel size 0.400.
:: Estimate normal with search radius 0.800.
Number of points to be removed:  19024
:: Compute FPFH feature with search radius 2.000.


In [33]:
def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
         3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(10000000, 0.999))
    return result

In [34]:
result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
print(result_ransac)


:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.400,
   we use a liberal distance threshold 0.600.
RegistrationResult with fitness=1.331182e-02, inlier_rmse=3.913275e-01, and correspondence_set size of 33
Access transformation to get result.


In [35]:
draw_registration_result(source, target, result_ransac.transformation)

In [36]:
print(result_ransac.transformation)

[[ 0.84966647 -0.49546606  0.18050006  0.14113014]
 [ 0.49105875  0.86818149  0.07156959 -0.72197555]
 [-0.19216712  0.02782585  0.98096765  0.56450302]
 [ 0.          0.          0.          1.        ]]


In [37]:
def refine_registration(source, target, source_fpfh, target_fpfh, voxel_size):
    radius_normal = voxel_size * 2
    distance_threshold = voxel_size * 0.4
    print(":: Point-to-plane ICP registration is applied on original point")
    print("   clouds to refine the alignment. This time we use a strict")
    print("   distance threshold %.3f." % distance_threshold)
    target.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    source.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    # orient normals so the ground is facing the same way
    target.orient_normals_to_align_with_direction()
    source.orient_normals_to_align_with_direction()
    # remove all points with a z normal component greater than the threshold
    target = remove_ground_points(target, 0.3)
    source = remove_ground_points(source, 0.3)
    result = o3d.pipelines.registration.registration_icp(
        source_down, target_down, distance_threshold, result_ransac.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    return result

In [38]:
result_icp = refine_registration(source, target, source_fpfh, target_fpfh,
                                 voxel_size)
print(result_icp)

:: Point-to-plane ICP registration is applied on original point
   clouds to refine the alignment. This time we use a strict
   distance threshold 0.160.
Number of points to be removed:  74512
Number of points to be removed:  63969
RegistrationResult with fitness=1.427585e-01, inlier_rmse=1.182929e-01, and correspondence_set size of 2482
Access transformation to get result.


In [39]:
np.set_printoptions(suppress=True)
print(result_icp.transformation)

[[ 0.85177071 -0.49223424  0.17942159 -0.04276647]
 [ 0.48721721  0.87012522  0.0741721  -0.69995257]
 [-0.1926293   0.02423966  0.98097217  0.6102065 ]
 [ 0.          0.          0.          1.        ]]


In [40]:
draw_registration_result(source, target, result_icp.transformation)

In [41]:
before_snow = o3d.io.read_point_cloud("/home/zak/senior_design/ros_livox/maps/fri_mar_12/all_points.pcd")
after_snow = o3d.io.read_point_cloud("/home/zak/senior_design/ros_livox/maps/mon_mar_15_after_snow_2/all_points.pcd")

draw_registration_result(before_snow, after_snow, result_icp.transformation)

In [42]:
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud],
                                      zoom=0.3412,
                                      front=[0.4257, -0.2125, -0.8795],
                                      lookat=[2.6172, 2.0475, 1.532],
                                      up=[-0.0694, -0.9768, 0.2024])

In [43]:
after_snow_down = after_snow.voxel_down_sample(voxel_size=0.02)
o3d.visualization.draw_geometries([after_snow_down],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

In [44]:
# use radius outlier to clean up the all points considerably
after_snow_out, ind = after_snow_down.remove_radius_outlier(nb_points=10, radius=0.06)
#display_inlier_outlier(uni_down_pcd, ind)

In [45]:
# draw the results of our new cleaned point cloud
# note: there is some data loss, the image greatly improves in clarity
o3d.visualization.draw_geometries([after_snow_out],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

In [46]:
# now we run the same process on the before snow cloud
before_snow_down = before_snow.voxel_down_sample(voxel_size=0.02)
before_snow_out, ind = before_snow_down.remove_radius_outlier(nb_points=10, radius=0.06)
o3d.visualization.draw_geometries([before_snow_out],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

In [21]:
# compare the two cleaned clouds
draw_registration_result(before_snow_out, after_snow_out, result_icp.transformation)

In [47]:
# apply calculated transformations and output the all points clouds for m3c2 distance calculation
before_snow_out = copy.deepcopy(before_snow_out).transform(result_icp.transformation)
o3d.io.write_point_cloud("before_aligned.pcd", before_snow_out)
o3d.io.write_point_cloud("after_aligned.pcd", after_snow_out)

True

In [48]:
# check to make sure our new files are transformed correctly with an identity matrix transform
before_aligned = o3d.io.read_point_cloud("before_aligned.pcd")
after_aligned = o3d.io.read_point_cloud("after_aligned.pcd")
draw_registration_result(before_aligned, after_aligned, np.identity(4))

In [51]:
# run cloudcompare in BASH command
# This assumes cloud compare was installed using the SNAPD package
os.system('cloudcompare.CloudCompare -O "before_aligned.pcd" -O "after_aligned.pcd" -M3C2 "m3c2_params.txt" -FILTER_SF 0.06 100 -C_EXPORT_FMT "PCD"')

0

In [ ]:
distances = after_snow_out.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 1.5 * avg_dist

after_snow_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    after_snow_out, o3d.utility.DoubleVector([radius, radius * 2]))
o3d.visualization.draw_geometries([after_snow_mesh])

In [22]:
radius_normal = voxel_size * 2
# estimate normals
after_snow_out.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))